# Transfer Learning

In [2]:
import matplotlib.pyplot as plt
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator,\
array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import RMSprop

The idea behind transfer learning is simply to make use of some pre-trained model to make predictions, rather than building a new model from scratch.

The plain fact is that several very powerful image-processing networks have already been built and perfected by scientists who have detailed knowledge about how all the layers of their models work. Moreover, many successful models have been trained on hundreds of thousands if not millions of images, and so they could be used for your images as well.

In general, the target will of course be different from the original that was used in training the model in the first place. But the idea is that the model will be good at picking up on the *deep features* of images, and so we can use *most* of the pre-trained model, in order to extract those deep features, and then just stick on a couple extra layers at the end that are appropriate for the data we have.

In what follows here we'll try building a network from scratch on some chest X-ray data. And then we'll see if we can get better accuracy by using [Imagenet](https://qz.com/1034972/the-data-that-changed-the-direction-of-ai-research-and-possibly-the-world/), a leading CNN for image recognition.

## CNN from Scratch

Let's look at some X-rays of lungs!

In [15]:
train_f = './chest_xray/train/'
test_f = './chest_xray/test/'
val_f = './chest_xray/val'

Keras's ImageDataGenerator can convert images (we have JPEGs here) to tensors of visual information!

In [3]:
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_f, 
        target_size=(64, 64))

val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_f, 
        target_size=(64, 64))

train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_f, 
        target_size=(64, 64))

Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.


ImageDataGenerator uses *data augmentation*, which means that it will take each image and transform it in various ways, ultimately using *only these transformations* as training data. [Here's](https://www.pyimagesearch.com/2019/07/08/keras-imagedatagenerator-and-data-augmentation/) a nice resource on keras's `ImageDataGenerator`.

And [here](https://bair.berkeley.edu/blog/2019/06/07/data_aug/) is a page with more information about data augmentation.

In [4]:
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

In [25]:
train_images[0][0]

### Model Building

In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(32, (4, 4), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['acc'])

W1015 10:58:51.419598 4450960832 deprecation_wrapper.py:119] From /Users/gdamico/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1015 10:58:51.435205 4450960832 deprecation_wrapper.py:119] From /Users/gdamico/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1015 10:58:51.438150 4450960832 deprecation_wrapper.py:119] From /Users/gdamico/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1015 10:58:51.454560 4450960832 deprecation_wrapper.py:119] From /Users/gdamico/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1015 10:58:51.5302

In [6]:
history_log = model.fit(train_images,
                    train_labels,
                    epochs=10,
                    batch_size=32,
                    validation_data=(test_images, test_labels))

W1015 10:58:53.768422 4450960832 deprecation_wrapper.py:119] From /Users/gdamico/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 32 samples, validate on 32 samples
Epoch 1/20
32/32 [==============================] - 1s 18ms/step - loss: 0.7028 - acc: 0.2500 - val_loss: 0.6922 - val_acc: 0.5000
Epoch 2/20
32/32 [==============================] - 0s 3ms/step - loss: 0.6959 - acc: 0.3750 - val_loss: 0.6944 - val_acc: 0.4844
Epoch 3/20
32/32 [==============================] - 0s 3ms/step - loss: 0.6894 - acc: 0.6875 - val_loss: 0.6966 - val_acc: 0.4375
Epoch 4/20
32/32 [==============================] - 0s 3ms/step - loss: 0.6833 - acc: 0.7500 - val_loss: 0.6990 - val_acc: 0.4375
Epoch 5/20
32/32 [==============================] - 0s 3ms/step - loss: 0.6774 - acc: 0.7500 - val_loss: 0.7016 - val_acc: 0.4375
Epoch 6/20
32/32 [==============================] - 0s 3ms/step - loss: 0.6718 - acc: 0.7500 - val_loss: 0.7044 - val_acc: 0.4375
Epoch 7/20
32/32 [==============================] - 0s 4ms/step - loss: 0.6664 - acc: 0.7500 - val_loss: 0.7074 - val_acc: 0.4375
Epoch 8/20
32/32 [===========================

Note the acc and val_acc scores!

## Now with Transfer Learning!

In [7]:
from keras.applications import VGG19

This tool comes from the [Visual Geometry Group](http://www.robots.ox.ac.uk/~vgg/research/very_deep/).

In [8]:
cnn_base = VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(64, 64, 3)
                )

cnn_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [9]:
batch_size=8

In [10]:
def extract_features(directory, sample_amount):
    features = np.zeros(shape=(sample_amount, 2, 2, 512)) 
    labels = np.zeros(shape=(sample_amount))
    generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        directory, target_size=(64, 64), 
        batch_size=batch_size, 
        class_mode='binary')
    i=0
    for inputs_batch, labels_batch in generator:
        features_batch = cnn_base.predict(inputs_batch)
        features[i * batch_size: (i + 1) * batch_size] = features_batch 
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_amount:
            break
    return features, labels

In [11]:
train_features, train_labels = extract_features(train_f, 5216) 
validation_features, validation_labels = extract_features(val_f, 16) 
test_features, test_labels = extract_features(test_f, 624)

train_features = np.reshape(train_features, (5216, 2048))
validation_features = np.reshape(validation_features, (16, 2048))
test_features = np.reshape(test_features, (624, 2048))

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [12]:
train_features.shape

(5216, 2048)

In [13]:
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=2048))
model.add(Dense(1, activation='sigmoid'))

In [14]:
model.compile(optimizer=RMSprop(),
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(train_features, train_labels,
                    epochs=10,
                    batch_size=10,
                    validation_data=(test_features, test_labels))

Train on 5216 samples, validate on 624 samples
Epoch 1/20
5216/5216 [==============================] - 2s 297us/step - loss: 0.2218 - acc: 0.9112 - val_loss: 1.0821 - val_acc: 0.7115
Epoch 2/20
5216/5216 [==============================] - 1s 256us/step - loss: 0.1395 - acc: 0.9438 - val_loss: 1.3907 - val_acc: 0.6843
Epoch 3/20
5216/5216 [==============================] - 1s 228us/step - loss: 0.1282 - acc: 0.9482 - val_loss: 0.5981 - val_acc: 0.8237
Epoch 4/20
5216/5216 [==============================] - 1s 248us/step - loss: 0.1125 - acc: 0.9567 - val_loss: 1.3986 - val_acc: 0.6939
Epoch 5/20
5216/5216 [==============================] - 1s 280us/step - loss: 0.1111 - acc: 0.9584 - val_loss: 0.6225 - val_acc: 0.8237
Epoch 6/20
5216/5216 [==============================] - 2s 290us/step - loss: 0.1038 - acc: 0.9617 - val_loss: 0.8248 - val_acc: 0.7548
Epoch 7/20
5216/5216 [==============================] - 1s 267us/step - loss: 0.0988 - acc: 0.9661 - val_loss: 1.1544 - val_acc: 0.7388
E

## Explore
What other networks are available inside keras?

In [26]:
from keras.applications import *

In [27]:
# Exercise: Use transfer learning with another pre-trained CNN on these data.
# See if you can improve on our metrics.